In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 935.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=28dc9e20a4cb5ff483d68f5a0b023cef0f657800b675c9e92038da9e06b0c64b
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
from sentence_transformers import SentenceTransformer, util
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
train_df = pd.read_csv('/kaggle/input/quora-question-pairs/train.csv.zip') 
test_df = pd.read_csv('/kaggle/input/quora-question-pairs/test.csv')

In [5]:
train_df.shape, test_df.shape

((404290, 6), (2345796, 3))

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [7]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [8]:
print(train_df.isnull().sum())
train_df=train_df.dropna()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64


In [9]:
len(train_df['question1'].values)

404287

In [10]:
len(train_df['question2'].values)

404287

In [11]:
corpus1=train_df['question1'].to_list()
corpus2=train_df['question2'].to_list()

In [12]:
corpus1[0:4]

['What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?',
 'Why am I mentally very lonely? How can I solve it?']

In [13]:
corpus2[0:5]

['What is the step by step guide to invest in share market?',
 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?',
 'How can Internet speed be increased by hacking through DNS?',
 'Find the remainder when [math]23^{24}[/math] is divided by 24,23?',
 'Which fish would survive in salt water?']

In [14]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

**Encoding**

In [15]:
embeddings1 = model.encode(corpus1, convert_to_tensor=True)
embeddings2 = model.encode(corpus2, convert_to_tensor=True)

Batches:   0%|          | 0/12634 [00:00<?, ?it/s]

Batches:   0%|          | 0/12634 [00:00<?, ?it/s]

In [16]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


**Cosine Simmilarities**

In [17]:
cosine_sim=[]

for i in range(len(embeddings1)):
    score=util.cos_sim(embeddings1[i],embeddings2[i])
    cosine_sim.append(score[0].item())

**Feature Engineering**

In [18]:
train_df['cosine_sim']=cosine_sim

**Sample -5**

In [19]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,cosine_sim
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.912277
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.655141
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.515561
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.104022
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.325348


**Defining a function to detecting duplicate using cosine value**

In [20]:
def cosine_to_pred(cosine_sim, threshold):
    if cosine_sim < threshold:
        label=0
    else:
        label=1
    return label

In [21]:
train_df['pred']=train_df['cosine_sim'].apply(lambda x: cosine_to_pred(x,0.8))

In [22]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,cosine_sim,pred
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.912277,1
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.655141,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.515561,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.104022,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.325348,0


**Find Out Accuracy**

In [23]:
acc=(train_df['pred']== train_df['is_duplicate']).mean()
print(acc)

0.7808635944267315


**Test Data Prediction**

In [24]:
test_df.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [25]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345796 entries, 0 to 2345795
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   test_id    int64 
 1   question1  object
 2   question2  object
dtypes: int64(1), object(2)
memory usage: 53.7+ MB


In [26]:
test_corpus1=test_df['question1'].to_list()
test_corpus2=test_df['question2'].to_list()

**Encoding the corpus in test Dataset**

In [27]:
test_embeddings1 = model.encode(test_corpus1, convert_to_tensor=True)
test_embeddings2 = model.encode(test_corpus2, convert_to_tensor=True)

Batches:   0%|          | 0/73307 [00:00<?, ?it/s]

Batches:   0%|          | 0/73307 [00:00<?, ?it/s]

**Cosine Similarities**

In [28]:
cosine_sim=[]

for i in range(len(test_embeddings1)):
    score=util.cos_sim(test_embeddings1[i],test_embeddings2[i])
    cosine_sim.append(score[0].item())

In [29]:
test_df['cosine_sim']=cosine_sim

In [30]:
test_df['pred']=test_df['cosine_sim'].apply(lambda x: cosine_to_pred(x, 0.8))

**Prediction On test dataset**

In [31]:
test_df.head()

,test_id,question1,question2,cosine_sim,pred
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,0.557410,0
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,0.815193,1
2,2,What but is the best way to send money from Ch...,What you send money to China?,0.813327,1
3,3,Which food not emulsifiers?,What foods fibre?,0.498818,0
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,0.544112,0
